In [ ]:
# load libraries
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import pathlib
import pandas as pd
import umap

from tensorflow.python.data.ops import dataset_ops
from tensorflow.python.keras.layers.preprocessing import image_preprocessing
from tensorflow.python.keras.preprocessing import dataset_utils
from tensorflow.python.ops import image_ops
from tensorflow.python.ops import io_ops
from tensorflow.keras import backend as K
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score, silhouette_score, calinski_harabasz_score
from sklearn.cluster import KMeans
from scipy.optimize import linear_sum_assignment as linear_assignment

The following code, taken from GitHub, allows you to match predictions with file names.

In [ ]:
WHITELIST_FORMATS = ('.bmp', '.gif', '.jpeg', '.jpg', '.png')

# Tensorflow override method to return fname as list as well as dataset
def image_dataset_from_directory(directory,
                                 labels='inferred',
                                 label_mode='int',
                                 class_names=None,
                                 color_mode='rgb',
                                 batch_size=32,
                                 image_size=(256, 256),
                                 shuffle=True,
                                 seed=None,
                                 validation_split=None,
                                 subset=None,
                                 interpolation='bilinear',
                                 follow_links=False):
  
  if labels != 'inferred':
    if not isinstance(labels, (list, tuple)):
      raise ValueError(
          '`labels` argument should be a list/tuple of integer labels, of '
          'the same size as the number of image files in the target '
          'directory. If you wish to infer the labels from the subdirectory '
          'names in the target directory, pass `labels="inferred"`. '
          'If you wish to get a dataset that only contains images '
          '(no labels), pass `label_mode=None`.')
    if class_names:
      raise ValueError('You can only pass `class_names` if the labels are '
                       'inferred from the subdirectory names in the target '
                       'directory (`labels="inferred"`).')
  if label_mode not in {'int', 'categorical', 'binary', None}:
    raise ValueError(
        '`label_mode` argument must be one of "int", "categorical", "binary", '
        'or None. Received: %s' % (label_mode,))
  if color_mode == 'rgb':
    num_channels = 3
  elif color_mode == 'rgba':
    num_channels = 4
  elif color_mode == 'grayscale':
    num_channels = 1
  else:
    raise ValueError(
        '`color_mode` must be one of {"rbg", "rgba", "grayscale"}. '
        'Received: %s' % (color_mode,))
  interpolation = image_preprocessing.get_interpolation(interpolation)
  dataset_utils.check_validation_split_arg(
      validation_split, subset, shuffle, seed)

  if seed is None:
    seed = np.random.randint(1e6)
  image_paths, labels, class_names = dataset_utils.index_directory(
      directory,
      labels,
      formats=WHITELIST_FORMATS,
      class_names=class_names,
      shuffle=shuffle,
      seed=seed,
      follow_links=follow_links)

  if label_mode == 'binary' and len(class_names) != 2:
    raise ValueError(
        'When passing `label_mode="binary", there must exactly 2 classes. '
        'Found the following classes: %s' % (class_names,))

  image_paths, labels = dataset_utils.get_training_or_validation_split(
      image_paths, labels, validation_split, subset)

  dataset = paths_and_labels_to_dataset(
      image_paths=image_paths,
      image_size=image_size,
      num_channels=num_channels,
      labels=labels,
      label_mode=label_mode,
      num_classes=len(class_names),
      interpolation=interpolation)
  #if shuffle:
    # Shuffle locally at each iteration
  #  dataset = dataset.shuffle(buffer_size=batch_size * 8, seed=seed)
  dataset = dataset.batch(batch_size)
  # Users may need to reference `class_names`.
  dataset.class_names = class_names
  return dataset, image_paths

def paths_and_labels_to_dataset(image_paths,
                                image_size,
                                num_channels,
                                labels,
                                label_mode,
                                num_classes,
                                interpolation):
  """Constructs a dataset of images and labels."""
  # TODO(fchollet): consider making num_parallel_calls settable
  path_ds = dataset_ops.Dataset.from_tensor_slices(image_paths)
  img_ds = path_ds.map(
      lambda x: path_to_image(x, image_size, num_channels, interpolation))
  if label_mode:
    label_ds = dataset_utils.labels_to_dataset(labels, label_mode, num_classes)
    img_ds = dataset_ops.Dataset.zip((img_ds, label_ds))
  return img_ds


def path_to_image(path, image_size, num_channels, interpolation):
  img = io_ops.read_file(path)
  img = image_ops.decode_image(
      img, channels=num_channels, expand_animations=False)
  img = image_ops.resize_images_v2(img, image_size, method=interpolation)
  img.set_shape((image_size[0], image_size[1], num_channels))
  return img

The following ad-hoc function calculates the "clustering" accuracy.

In [ ]:
# auxiliary function
def accuracy(y_true, y_pred):
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    row_ind, col_ind = linear_assignment(w.max() - w)
    
    return w[row_ind, col_ind].sum() * 1.0 / y_pred.size

In [ ]:
# load data
data_dir = 'PATH_TO_DATA'

data_dir = pathlib.Path(data_dir)
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

batch_size = 32
img_height = 224
img_width = 224

ds, paths = image_dataset_from_directory(
    data_dir,
    shuffle=True,
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size,  
    label_mode='int')

class_names = ds.class_names
print(class_names)

# display a few images
plt.figure(figsize=(10, 10))
for images, labels in ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        print(paths[i])
        plt.axis("off")

The following code extracts features from a pre-trained model.

In [ ]:
preprocess_input = tf.keras.applications.densenet.preprocess_input

base_model = tf.keras.applications.DenseNet121(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet',
                                               pooling='avg')

base_model.trainable = False

inputs = tf.keras.Input(shape=(224, 224, 3))
x = preprocess_input(inputs)
outputs = base_model(x, training=False)
model = tf.keras.Model(inputs, outputs)

model.summary()

features = []
labels =  np.array([])
for x, y in ds.take(247): # the input argument decides the number of data batches you want to consider, e.g. 247
    features.append(model.predict(x))
    labels = np.concatenate([labels, y])

features = np.asarray(features).reshape(247*32, 1024)

The following snippet defines and pre-trains a simple fully-connected autoencoder.

In [ ]:
def autoencoder(dims, act='relu', init='glorot_uniform'):
    n_stacks = len(dims) - 1
    # input
    input_img = tf.keras.layers.Input(shape=(dims[0],), name='input')
    x = input_img
    # internal layers of the encoder
    for i in range(n_stacks-1):
        x = tf.keras.layers.Dense(dims[i + 1], activation=act, kernel_initializer=init, name='encoder_%d' % i)(x)

    # bottleneck
    encoded = tf.keras.layers.Dense(dims[-1], kernel_initializer=init, name='encoder_%d' % (n_stacks - 1))(x)  # hidden layer, features are extracted from here

    x = encoded
    # hidden layers of the decoder
    for i in range(n_stacks-1, 0, -1):
        x = tf.keras.layers.Dense(dims[i], activation=act, kernel_initializer=init, name='decoder_%d' % i)(x)

    # output
    x = tf.keras.layers.Dense(dims[0], kernel_initializer=init, name='decoder_0')(x)
    decoded = x
    return tf.keras.models.Model(inputs=input_img, outputs=decoded, name='AE'), tf.keras.models.Model(inputs=input_img, outputs=encoded, name='encoder')

dims = [1024, 500, 500, 2000, 10]
init = tf.keras.initializers.VarianceScaling(scale=1. / 3., mode='fan_in',
                                             distribution='uniform')
autoencoder, encoder = autoencoder(dims, init=init)
autoencoder.summary()

autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(features, features, batch_size=256, epochs=200)

# save model
save_dir = 'PATH_TO_MODELS'
autoencoder.save(save_dir + 'pretrained_autoencoder.h5')

In [ ]:
The following is the original implementation of the deep clustering layer.

In [ ]:
n_clusters = 10 # number of clusters to be considered

class ClusteringLayer(tf.keras.layers.Layer):
    def __init__(self, n_clusters, weights=None, alpha=1.0, **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(ClusteringLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.initial_weights = weights
        self.input_spec = tf.keras.layers.InputSpec(ndim=2)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        self.input_spec = tf.keras.layers.InputSpec(dtype=K.floatx(), shape=(None, input_dim))
        self.clusters = self.add_weight('clusters', (self.n_clusters, input_dim), initializer='glorot_uniform')
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    def call(self, inputs, **kwargs):
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1)) # Make sure each sample's 10 values add up to 1.
        return q

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 2
        return input_shape[0], self.n_clusters

    def get_config(self):
        config = {'n_clusters': self.n_clusters}
        base_config = super(ClusteringLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

clustering_layer = ClusteringLayer(n_clusters, name='clustering')(encoder.output)
model = tf.keras.models.Model(inputs=encoder.input, outputs=clustering_layer)
model.summary()

The following snippet trains the model and shows its performance.

In [ ]:
model.compile(optimizer='adam', loss='kld')

# initialize cluster centroids using k-means
kmeans = KMeans(n_clusters=n_clusters, n_init=20)
y_pred = kmeans.fit_predict(encoder.predict(features))

y_pred_last = np.copy(y_pred)
model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])

# compute auxiliary target distribution
def target_distribution(q):
    weight = q ** 2 / q.sum(0)
    return (weight.T / weight.sum(1)).T

# hyperparameters
batch_size = 128
loss = 0
index = 0
maxiter = 8000
update_interval = 140
index_array = np.arange(features.shape[0])
tol = 0.001 # tolerance threshold to stop training

for ite in range(int(maxiter)):
    if ite % update_interval == 0:
        q = model.predict(features, verbose=0)
        p = target_distribution(q)  # update the auxiliary target distribution
        embedded_features = encoder.predict(features) # embeddings
        
        # evaluate clustering performance
        y_pred = q.argmax(1)
        if labels is not None:
            acc = np.round(accuracy(labels, y_pred), 5)
            nmi = np.round(normalized_mutual_info_score(labels, y_pred), 5)
            ars = np.round(adjusted_rand_score(labels, y_pred), 5)
            ss = np.round(silhouette_score(embedded_features, y_pred), 5)
            chs = np.round(calinski_harabasz_score(embedded_features, y_pred), 5)
            loss = np.round(loss, 5)
            print('Iter %d: acc = %.5f, nmi = %.5f, ars = %.5f, ss = %.5f, chs = %.5f' % (ite, acc, nmi, ars, ss, chs), '; loss=', loss)

        # check stopping criterion
        delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
        y_pred_last = np.copy(y_pred)
        if ite > 0 and delta_label < tol:
            print('delta_label ', delta_label, '< tol ', tol)
            print('Reached tolerance threshold. Stopping training.')
            break
            
    idx = index_array[index * batch_size: min((index+1) * batch_size, features.shape[0])]
    loss = model.train_on_batch(x=features[idx], y=p[idx])
    index = index + 1 if (index + 1) * batch_size <= features.shape[0] else 0

The last part of the code is responsible for the qualitative evaluation of the clusters found.

In [ ]:
# t-SNE visualization
tsne = TSNE(n_components=2, random_state=42)
tsne_embedding = tsne.fit_transform(embedded_features)

plt.figure(dpi=600)
plt.scatter(*zip(*tsne_embedding[:,:2]), c=y_pred, cmap='viridis')
plt.tick_params(labelsize=15)
#plt.axis("off")
plt.show()

# UMAP visualization
reducer = umap.UMAP()
umap_embedding = reducer.fit_transform(embedded_features)

plt.figure(dpi=600)
plt.scatter(*zip(*umap_embedding[:,:2]), c=y_pred, cmap='viridis')
plt.tick_params(labelsize=15)
#plt.axis('off')
plt.show()

In [ ]:
sample_paths = paths

sample_df = pd.DataFrame(sample_paths, columns=['path'])
y_pred_df = pd.DataFrame(y_pred, columns=['y_pred'])
df = pd.concat([sample_df, y_pred_df], axis=1)

cluster = 1 # clyster to be checked
sample = df[df['y_pred']==cluster].sample(n=50, random_state=42)

# display a few images from that cluster
for index, row in sample.iterrows():
    print(row['path'])
    img = mpimg.imread(row['path'])
    imgplot = plt.imshow(img)
    plt.axis("off")
    plt.show()